In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import imutils
import pickle
import easyocr
import time
import cvzone
import random
import os
from datetime import datetime
width=107
height=45
v=[]
t11=[]
t22=[]
p=[]
b=[]
j=1

def img1():
    path = r"C:\Users\sudha\cars"
    img2.random_filename = random.choice([x for x in os.listdir(path)
                                     if os.path.isfile(os.path.join(path, x))])
    flag=0
    for i in v:
        if i in v:
            flag+=1
    if flag==3:
        print("\n\n\nParking for vehicles is full!!!\n\n\n")
        Key()
    else:
        img2()

def img2():
    if img2.random_filename in v:
        img1()
    else:
        v.append(img2.random_filename)
        now = datetime.now()

        t1 = now.strftime("%H:%M:%S")

        img2.hh, img2.mm, img2.ss = t1.split((':'))
        tim_start=int(img2.hh) * 3600 + int(img2.mm) * 60 + int(img2.ss)
        t11.append(tim_start)
        t22.append(0)
        print("\nCar entered : ",img2.random_filename)
        print("\nStarting time of car  :",img2.hh,":",img2.mm,":", img2.ss)
        print('\n')


def img4():
   
    path = r"C:\Users\sudha\cars"
    img2.random_filename = random.choice([x for x in os.listdir(path)
                                     if os.path.isfile(os.path.join(path, x))])
    img2()

    img=cv2.imread(img2.random_filename)
    #plt.imshow(cv2.cvtColor(img,cv2.COLOR_BGR2RGB))
    GrayImg=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    filters=cv2.bilateralFilter(GrayImg,11,17,17)
    edged=cv2.Canny(filters,30,200)
    #plt.imshow(cv2.cvtColor(edged,cv2.COLOR_BGR2RGB))

    keypoints=cv2.findContours(edged.copy(),cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    contours=imutils.grab_contours(keypoints)
    contours=sorted(contours,key=cv2.contourArea,reverse=True)[:10]

    location=None

    for contour in contours:
        approx=cv2.approxPolyDP(contour,10,True)

        if(len(approx)==4):
            location=approx
            break
    print(location,"\n")
    mask=np.zeros(GrayImg.shape,np.uint8)

    New_Img=cv2.drawContours(mask,[location],0,255,-1)

    New_Img=cv2.bitwise_and(img,img,mask=mask)

    #plt.imshow(cv2.cvtColor(New_Img,cv2.COLOR_BGR2RGB))

    (x,y)=np.where(mask==255)

    (x1,y1)=(np.min(x),np.min(y))
    (x2,y2)=(np.max(x),np.max(y))
    Img_crop=img[x1:x2+1,y1:y2+1]

    #plt.imshow(cv2.cvtColor(Img_crop,cv2.COLOR_BGR2RGB))

    reader=easyocr.Reader(['en'])
    result=reader.readtext(Img_crop)   
    print(result,"\n")
    text=result[0][-2]
    print("Number plate :"+text)
    print("Data entered successfully")
    font=cv2.FONT_HERSHEY_SIMPLEX
    img2.res=cv2.putText(img,text=text,org=(approx[0][0][0],approx[2][0][1]+40),fontFace=font,fontScale=0.6,color=(0,255,0), thickness=2)
    img2.res=cv2.rectangle(img,tuple(approx[0][0]),tuple(approx[2][0]),(0,255,0),2)
    plt.imshow(cv2.cvtColor(img2.res,cv2.COLOR_BGR2RGB))
        
        
        
def display():
 
    cam=cv2.VideoCapture('carPark.mp4')

    with open('CarParkPos24','rb') as f:
            posList=pickle.load(f)
    def CheckPark(img1,cnt):
        count_space=0
        count_oc=0
        for pos in posList:
            x,y=pos
               
            cnt+=j
            imgCrop=img1[y:y+height,x:x+width]
            count=cv2.countNonZero(imgCrop)
            cvzone.putTextRect(img,str(count),(x,y+height-5),scale=1,thickness=2,offset=0,colorR=(255,0,255))
            if count<1000:
                
                color=(0,255,0)
                thickness=2
                count_space+=1
            else:
                color=(0,0,255)
                thickness=2
                count_oc+=1
            cvzone.putTextRect(img,str(cnt),(x+10,y+15),scale=1,thickness=1,offset=2,colorR=(0,0,0))
            cv2.rectangle(img,pos,(pos[0]+width,pos[1]+height),color,thickness)    
        cvzone.putTextRect(img,f'VACANT {count_space}/{len(posList)}',(20,50),scale=2,thickness=3,offset=10,colorR=(0,255,0))
        cvzone.putTextRect(img,f'SystemParkCounter',(750,50),scale=2,thickness=2,offset=10,colorR=(0,0,0))
        cvzone.putTextRect(img,f'A',(90,710),scale=2,thickness=2,offset=10,colorR=(0,0,0))
        cvzone.putTextRect(img,f'B',(190,710),scale=2,thickness=2,offset=10,colorR=(0,0,0))
        cvzone.putTextRect(img,f'C',(450,710),scale=2,thickness=2,offset=10,colorR=(0,0,0))
        cvzone.putTextRect(img,f'D',(555,710),scale=2,thickness=2,offset=10,colorR=(0,0,0))
        cvzone.putTextRect(img,f'E',(800,710),scale=2,thickness=2,offset=10,colorR=(0,0,0))
        cvzone.putTextRect(img,f'F',(950,710),scale=2,thickness=2,offset=10,colorR=(0,0,0))
    while True:
        if cam.get(cv2.CAP_PROP_POS_FRAMES)==cam.get(cv2.CAP_PROP_FRAME_COUNT):
            cam.set(cv2.CAP_PROP_POS_FRAMES,0)

        success,img=cam.read()
        GrayImg=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)     # ImgGray=0.2989*R+0.5870*G+0.1140*B
        GausBlur=cv2.GaussianBlur(GrayImg,(3,3),1)
        ImgThresh=cv2.adaptiveThreshold(GausBlur,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY_INV,25,16)  #img,max value,method,inverse,values
        ImgMedian=cv2.medianBlur(ImgThresh,5)

        kernel=np.ones((4,4),np.uint8)
        imgDilate=cv2.dilate(ImgMedian,kernel,iterations=1)
        CheckPark(imgDilate,1)
        GrayImg=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        # plt.imshow(cv2.cvtColor(GrayImg,cv2.COLOR_BGR2RGB))
        filters=cv2.bilateralFilter(GrayImg,11,17,17)
        edged=cv2.Canny(filters,30,200)

        
        cv2.imshow("Image",img)

        if(cv2.waitKey(1)==27):
            break
    cv2.destroyAllWindows()
    
    
    

def exit_car():
    cost=10
    from datetime import datetime
    ram=random.choice(v)
    
    print("\n\n~~~~~~~~~THANK YOU~~~~~~~~~~\n")
    print("\nCar Exit :",ram)

    now = datetime.now()
    t2 = now.strftime("%H:%M:%S")
    
    hh1, mm1, ss1 = t2.split((':'))
    tim_end=int(hh1) * 3600 + int(mm1) * 60 + int(ss1)
    
    
    inx=v.index(ram)

    t22[inx]=tim_end

    print("\nIndex of ",ram," is ",inx)
    print("\nEntry time of car",img2.hh,":" ,img2.mm,":", img2.ss)
    print("\nExit time of car",hh1,":", mm1,":", ss1)
    
    
    
    t=int(t22[inx]-t11[inx])

    t = int(t % (24 * 3600))
    hour = int(t // 3600)
    t %= 3600
    minutes =int( t // 60)
    t %= 60
    print("\nDifference of time is  :",t)
    if(minutes<1):
        cost=cost
    if(minutes>=1):
        cost=15
    if(minutes>=2):
        cost=20
    if(minutes>=3):
        cost=50
    print("\nThe time consumed is :" ,hour,":", minutes,":", t)
    print("\nThe cost of parking is :" ,cost)
    
    v.pop(inx)
    print("\nRemaining cars in the parking area  :" ,v)
    t11.pop(inx)
    t22.pop(inx)
    
    
print("~~~~~~~~~~CAMSION WELCOME'S YOU~~~~~~~~~~~~~~  ")

def Key():
    n=int(input("Enter 1 to Scan and 2 to Exit :"))
    if n==1:
        img4()
        display()
        Key()
    if n==2:
        if len(v)==0:
            print("\n\n~~~~~No car in the parking slots!!!\n\n")
            Key()
        else:
            exit_car()
            Key()
def password():
    
    m=int(input("Enter the PASSWORD :"))
    if m==1234:
        Key()
    else:
        b.append(1)

        if len(b)==2 or len(b)==5:
            print("\n\n\nCONTACT THE ENGINEER FOR THE PASSWORD!!!!")
        password() 
password()



def Key():
        n=int(input("Enter 1 to Scan and 2 to Exit :"))
        if n==1:
            img4()
            display()
            plt.imshow(cv2.cvtColor(img2.res,cv2.COLOR_BGR2RGB))
            Key()
        elif n==2:
            if len(v)==0:
                print("\n\n~~~~~No car in the parking slot+s!!!\n\n")
                Key()
            else:
                return 0